In [99]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [100]:
# read in Titanic training and test datasets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


The training set has 891 examples and 11 features + target variable (Survived):

In [101]:
train_df.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

Only 38% of the training sert survived the Titanic.

In [102]:
train_df.head(8)

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   
5            6         0       3   
6            7         0       1   
7            8         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   
5                                   Moran, Mr. James    male   NaN      0   
6                            McCarthy, Mr. Timothy J    male  54.0      0   
7                     Palsson, Master. Gosta Leonard    male   2.0      3   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
5      0            330877   8.4583   NaN        Q  
6      0             17463  51.8625   E46        S  
7      1            349909  21.0750   NaN        S

We can see that there are some missing data (see Cabin). Let's look a little more closely.

In [103]:
# count up the missing data in training and test set
total=train_df.isnull().sum().sort_values(ascending=False)
percent_1 = train_df.isnull().sum()/len(train_df)*100
percent_2 = (round(percent_1,1)).sort_values(ascending=False)
missing_data=pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

Total     %
Cabin       687  77.1
Age         177  19.9
Embarked      2   0.2
Fare          0   0.0
Ticket        0   0.0

In [104]:
total=test_df.isnull().sum().sort_values(ascending=False)
percent_1 = test_df.isnull().sum()/len(test_df)*100
percent_2 = (round(percent_1,1)).sort_values(ascending=False)
missing_data=pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

Total     %
Cabin       327  78.2
Age          86  20.6
Fare          1   0.2
Embarked      0   0.0
Ticket        0   0.0

The Embarked feature has only 2 missing values, which can be easily filled or their samples removed. The Age feature has almost 20% missing data. The Cabin feature is missing over 77% of the data. Should we drop Cabin? What else should we drop? Name? PassengerId?

In [105]:
# drop features
train_df=train_df.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], axis=1)
test_df=test_df.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], axis=1)

# create a mapping between gender and numerical values
sex_mapping = {'male':0, 'female':1}
# combine train and test data
data = [train_df, test_df]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)
    
for dataset in data:
    dataset['Sex'] = dataset['Sex'].astype('category')

In [106]:
train_df.head()

Survived  Pclass Sex   Age  SibSp  Parch     Fare Embarked
0         0       3   0  22.0      1      0   7.2500        S
1         1       1   1  38.0      1      0  71.2833        C
2         1       3   1  26.0      0      0   7.9250        S
3         1       1   1  35.0      1      0  53.1000        S
4         0       3   0  35.0      0      0   8.0500        S

In [107]:
test_df.head()

Pclass Sex   Age  SibSp  Parch     Fare Embarked
0       3   0  34.5      0      0   7.8292        Q
1       3   1  47.0      1      0   7.0000        S
2       2   0  62.0      0      0   9.6875        Q
3       3   0  27.0      0      0   8.6625        S
4       3   1  22.0      1      1  12.2875        S

In [108]:
# impute the missing data
data = [train_df, test_df]

for dataset in data:
    dataset['Fare'].fillna(dataset['Fare'].mean(), inplace = True)
    dataset['Age'].fillna(dataset['Age'].mean(), inplace = True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
    
# count up the missing data
total_train=train_df.isnull().sum().sort_values(ascending=False)
total_test=test_df.isnull().sum().sort_values(ascending=False)
print(total_train)
print(total_test)

Embarked    0
Fare        0
Parch       0
SibSp       0
Age         0
Sex         0
Pclass      0
Survived    0
dtype: int64
Embarked    0
Fare        0
Parch       0
SibSp       0
Age         0
Sex         0
Pclass      0
dtype: int64


In [109]:
# create a mapping between ports and numerical values
port_mapping = {'S':0, 'C':1, 'Q':2}
# combine training and test data
data = [train_df, test_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].map(port_mapping)
    
for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].astype('category')

In [110]:
# categorize the age:
for datasets in data:
    datasets['Age'] = pd.cut(datasets['Age'],
                            bins = [0,12,50,200],
                            labels = ["Child","Adult","Elder"])

# check the categorization
datasets['Age'].head()

0    Adult
1    Adult
2    Elder
3    Adult
4    Adult
Name: Age, dtype: category
Categories (3, object): [Child < Adult < Elder]

In [111]:
# create a mapping between ports and numerical values
Age_mapping = {'Child':0, 'Adult':1, 'Elder':2}
# combine training and test data
data = [train_df, test_df]

for dataset in data:
    dataset['Age'] = dataset['Age'].map(Age_mapping)
    
for dataset in data:
    dataset['Age'] = dataset['Age'].astype('category')

In [112]:
# categorize the fare:

for datasets in data:
    datasets['Fare'] = pd.cut(datasets['Fare'],
                            bins = [-1,14.454,10000],
                            labels = ["Cheap","Expensive"])
    

# create a mapping between ports and numerical values
Fare_mapping = {'Cheap':int(0), 'Expensive':int(1)}
# combine training and test data
data = [train_df, test_df]

for dataset in data:
    dataset['Fare'] = dataset['Fare'].map(Fare_mapping)
    
for dataset in data:
    dataset['Fare'] = dataset['Fare'].astype('category')

In [113]:
# categorize the Pclass
for dataset in data:
    dataset['Pclass'] = dataset['Pclass'].astype('category')

In [114]:
# split training and test set
X = train_df.iloc[:,1:9]
y = train_df.iloc[:,0]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

In [115]:
from sklearn.svm import SVC

svm = SVC(kernel = 'rbf', C=1 , gamma = 0.01)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("Accuracy: ", 1 - (y_test != y_pred_svm).sum()/len(y_test))

Accuracy:  0.8022388059701493


In [116]:
# make prediction for test set
test_df['Survival'] = svm.predict(test_df)
test_df.head()

Pclass Sex Age  SibSp  Parch Fare Embarked  Survival
0      3   0   1      0      0    0        2         0
1      3   1   1      1      0    0        0         1
2      2   0   2      0      0    0        2         0
3      3   0   1      0      0    0        0         0
4      3   1   1      1      1    0        0         1